### Overview

This notebook measures the speed between

 * regular python function
 * numba function
 * numba with inline numba functions

We would use a simple tradebook example to test this.

In this example, we would be BUY a instrument when the price goes above the high price of a previous lookback period or SELL the instrument when the price goes below the low price

#### Downloading data

Download data to be used

In [1]:
import yfinance as yf
import numpy as np
from numba import njit

msft = yf.Ticker("MSFT")

# get all stock info
msft.info

# get historical market data
df = msft.history(period="max")
df.columns = [x.lower() for x in df.columns]
df.head()

,open,high,low,close,volume,dividends,stock splits
Date,,,,,,,
1986-03-13 00:00:00-05:00,0.054594,0.062623,0.054594,0.059946,1031788800,0.0,0.0
1986-03-14 00:00:00-05:00,0.059947,0.063158,0.059947,0.062087,308160000,0.0,0.0
1986-03-17 00:00:00-05:00,0.062087,0.063694,0.062087,0.063158,133171200,0.0,0.0
1986-03-18 00:00:00-05:00,0.063158,0.063694,0.061017,0.061552,67766400,0.0,0.0
1986-03-19 00:00:00-05:00,0.061552,0.062087,0.059947,0.060482,47894400,0.0,0.0


#### Test timing

We are creating a few simple functions to test the time difference between code directly written inside numba and calling a numba function inside numba 

In [2]:
def tradebook_long(close, date):
    """
    long only tradebook
    """
    length = len(close)
    side = 0
    entry_price = exit_price = 0
    entry_time = exit_time = date[0]
    for i in range(1, length):
        if side == 0 and close[i] > close[i - 1]:
            side = 1
            entry_price = close[i]
            entry_time = date[i]
    exit_price = close[-1]
    exit_time = date[-1]

    return side, entry_price, entry_time, exit_price, exit_time


def tradebook_short(close, date):
    """
    short only tradebook
    """
    length = len(close)
    side = 0
    entry_price = exit_price = 0
    entry_time = exit_time = date[0]
    for i in range(1, length):
        if side == 0 and close[i] < close[i - 1]:
            side = -1
            entry_price = close[i]
            entry_time = date[i]
    exit_price = close[-1]
    exit_time = date[-1]

    return side, entry_price, entry_time, exit_price, exit_time


def tradebook(close, date, start: int = 300):
    """
    both long and short tradebook
    """
    length = len(close)
    side = 0
    entry_price = exit_price = 0
    entry_time = exit_time = date[0]
    ref_high = np.max(close[:start])
    ref_low = np.min(close[:start])
    for i in range(start, length):
        if side == 0:
            if close[i] > ref_high:
                side = 1
                entry_price = close[i]
                entry_time = date[i]
            elif close[i] < ref_low:
                side = -1
                entry_price = close[i]
                entry_time = date[i]
    exit_price = close[i]
    exit_time = date[i]
    return side, entry_price, entry_time, exit_price, exit_time


# create the numba functions
ntradebook_long = njit(tradebook_long)
ntradebook_short = njit(tradebook_short)
ntradebook = njit(tradebook)

In [3]:
# Numba inline function
@njit
def tradebook_inline(close, date, start: int = 300):
    length = len(close)
    ref_high = np.max(close[:start])
    ref_low = np.min(close[:start])
    for i in range(start, length):
        if close[i] > ref_high:
            return ntradebook_long(close[i - 1 :], date[i - 1 :])
        elif close[i] < ref_low:
            return ntradebook_short(close[i - 1 :], date[i - 1 :])


tradebook_inline(df.close.values, df.index.values)

(1,
 0.27404147386550903,
 numpy.datetime64('1987-09-28T04:00:00.000000000'),
 416.32000732421875,
 numpy.datetime64('2024-10-11T04:00:00.000000000'))

In [4]:
# Print values to see if they are right and also to run the njit function once
print(tradebook_long(df.close.values, df.index.values))
print(tradebook_short(df.close.values, df.index.values))
print(tradebook(df.close.values, df.index.values))
print(ntradebook_long(df.close.values, df.index.values))
print(ntradebook_short(df.close.values, df.index.values))
print(ntradebook(df.close.values, df.index.values))
print(tradebook_inline(df.close.values, df.index.values))

(1, 0.062087323516607285, numpy.datetime64('1986-03-14T05:00:00.000000000'), 416.32000732421875, numpy.datetime64('2024-10-11T04:00:00.000000000'))
(-1, 0.061552081257104874, numpy.datetime64('1986-03-18T05:00:00.000000000'), 416.32000732421875, numpy.datetime64('2024-10-11T04:00:00.000000000'))
(1, 0.27404147386550903, numpy.datetime64('1987-09-28T04:00:00.000000000'), 416.32000732421875, numpy.datetime64('2024-10-11T04:00:00.000000000'))
(1, 0.062087323516607285, numpy.datetime64('1986-03-14T05:00:00.000000000'), 416.32000732421875, numpy.datetime64('2024-10-11T04:00:00.000000000'))
(-1, 0.061552081257104874, numpy.datetime64('1986-03-18T05:00:00.000000000'), 416.32000732421875, numpy.datetime64('2024-10-11T04:00:00.000000000'))
(1, 0.27404147386550903, numpy.datetime64('1987-09-28T04:00:00.000000000'), 416.32000732421875, numpy.datetime64('2024-10-11T04:00:00.000000000'))
(1, 0.27404147386550903, numpy.datetime64('1987-09-28T04:00:00.000000000'), 416.32000732421875, numpy.datetime64

In [5]:
# %timeit values to see if they are right and also to run the njit function once
%timeit tradebook_long(df.close.values, df.index.values)
%timeit tradebook_short(df.close.values, df.index.values)
%timeit tradebook(df.close.values, df.index.values)
%timeit ntradebook_long(df.close.values, df.index.values)
%timeit ntradebook_short(df.close.values, df.index.values)
%timeit ntradebook(df.close.values, df.index.values)
%timeit tradebook_inline(df.close.values, df.index.values)

508 µs ± 22.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
504 µs ± 12.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
580 µs ± 85.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
18.6 µs ± 271 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
18.9 µs ± 380 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
19.5 µs ± 96.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
19.6 µs ± 135 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


#### Inlining speed without loop

Here we would be testing numba function with a single step instead of a loop. This function would just return a True or False for every value in the array and whether numba would automatically inline this function

<div class="alert alert-block alert-success"> Man, this is stunning, loops are automatically inlined so you can write lot of one liners</div>

In [10]:
def loop_run(prices, val=1e6):
    length = len(prices)
    for i in range(length):
        # We know prices would always be less than val so the entire loop would be run
        if prices[i] > val:
            return prices[i]
    return prices[i]
assert round(loop_run(df.close.values),4) == round(df.close.iloc[-1],4)

In [14]:
@njit
def ncmp(x,val):
    return True if x > val else False

cmp = lambda x,val: True if x > val else False

ncmp(10,15)
%timeit cmp(10,15)
%timeit ncmp(10,15)

144 ns ± 2.58 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
300 ns ± 4.63 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [17]:
@njit
def inline_loop_run(prices, val=1e6):
    length = len(prices)
    for i in range(length):
        # We know prices would always be less than val so the entire loop would be run
        k = ncmp(prices[i], val)
        if k:
            return prices[i]
    return prices[i]
assert round(inline_loop_run(df.close.values),4) == round(df.close.iloc[-1],4)

In [18]:
nloop_run = njit(loop_run)
%timeit loop_run(df.close.values)
%timeit nloop_run(df.close.values)
%timeit inline_loop_run(df.close.values)

2.03 ms ± 94.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
17.6 µs ± 7.47 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
17.6 µs ± 6.22 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
